# IMPORT

In [1]:
import numpy as np
import pandas as pd

In [2]:
# custom username by pwd
import os
pwd = os.getcwd()
username = pwd[9:pwd.find('\\',9)]

In [3]:
# การแก้ Fuzzy ว่าเอากี่ Percent ให้แก้จรง cutoff = x
import difflib

def fuzzy_merge(df1, df2, left_on, right_on, how='inner', cutoff=0.90):
    df_other= df2.copy()
    df_other[left_on] = [get_closest_match(x, df1[left_on], cutoff)
                         for x in df_other[right_on]]
    return df1.merge(df_other, on=left_on, how=how)

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, cutoff=cutoff)
    return matches[0] if matches else None

In [4]:
def isEnglish(s):
  try:
    s.encode(encoding='utf-8').decode('ascii')
  except UnicodeDecodeError:
    return False
  else:
    return True

In [5]:
def clean_string(x):
  if pd.isna(x) == False:
    x = x.lower()
    x = x.strip()
    remove_list = [' ','ด็อกเตอร์','ดร.','ว่าที่','พล.อ.','พล.ท.','พล.ต.','พ.อ.','พ.ท.','พ.ต.','ร.อ.','ร.ท.','ร.ต.','จ.ส.อ.','จ.ส.ท.','จ.ส.ต','ส.อ.',
                           'ส.ท.','ส.ต.',
                          'พลฯ','นนร.','พล.ร.อ.','พล.ร.ท.','พล.ร.ต.','น.อ.','ร.น.','น.ท.','น.ต.','ร.อ.','ร.ท.','ร.ต.','พ.จ.อ.','พ.จ.ท.','พ.จ.ต.',
                          'จ.อ.','จ.ท.','จ.ต.','พล.อ.อ.','พล.อ.ท.','พล.อ.ต.','พ.อ.อ.','พ.อ.ท.','พ.อ.ต.','นนอ.','นจอ.',
                          'พล.ต.อ.','พล.ต.ท.','พล.ต.ต.','พ.ต.อ.','พ.ต.ท.','พ.ต.ต.','ร.ต.อ.','ร.ต.ท.','ร.ต.ต.','ด.ต.','จ.ส.ต.','ส.ต.อ.','ส.ต.ท.','ส.ต.ต.'
                           ,'พลเอก','พลโท','พลตรี','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','พลทหาร',
                        'พลเรือเอก','พลเรือโท','พลเรือตรี','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี',
                           'พลอากาศเอก' , 'พลอากาศโท','พลอากาศตรี','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท' ,'เรืออากาศตรี',
                           'พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี',
                           'mr.','mrs.','ms.','limited','inc.','co., ltd.,','ltd','(ไทย)','(ประเทศไทย)',
                        '','(',')','-','.',',','บริษัท','บจก','จำกัด','มหาชน','ห้างหุ้นส่วน','จำกัด','publiccompanylim','company','–','/' ,
                           'นาย','นางสาว' ,'นาง' , 'x0000' , '_' , 'และหรือย่อย']
    for i in range(len(remove_list)):
      x = x.replace(remove_list[i],'')
  return x

In [6]:
input_path = 'C:\\Users\\'+ username +'\\OneDrive - EXIM BANK OF THAILAND\\Shared Drive บส ตส\\Data\\ปี 2567 Q1\\'
output_path = 'C:\\Users\\'+ username +'\\OneDrive - EXIM BANK OF THAILAND\\Shared Drive บส ตส\\3. รวมศูนย์\\RESULT\\'


# input_path = r'C:\Users\nipittab\OneDrive - EXIM BANK OF THAILAND\Shared Drive บส ตส\Data\ปี 2566 Q3/'
# output_path = r'C:\Users\nipittab\OneDrive - EXIM BANK OF THAILAND\Shared Drive บส ตส\3. รวมศูนย์\RESULT/'

# INPUT

In [7]:
#@title ข้อมูล Outstanding
#จาก Share Point
outstanding = '03.วงเงินและยอดคงค้าง_มี.ค.2567.xlsx' #@param {type:'string'}
df_out = pd.read_excel(input_path + outstanding , dtype = str)

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [8]:
#@title ชื่อไฟล์ รหัสลูกค้า เลขนิติบุคคล
#จาก Jupyter โดยการ Merge ข้อมูล วงเงินยอดคงค้าง กับ ไฟล์ข้อมูลลูกค้าในระบบ AS400
# Link Share Point : http://sp.exim.go.th/dc/report/Report14/Forms/AllItems.aspx
id_cuscode = 'รหัส_เลขนิติ_Q1_67.xlsx' #@param {type:'string'}
df_pm = pd.read_excel(input_path + id_cuscode , dtype = str)

In [9]:
#@title ข้อมูล ECS File บัญชี
#ได้รับจากพี่หนุ่ม ภาคิน ทุกเดือน ฝ่าย พส.
ecs1 = 'ลูกค้าวงเงินหลักประกัน_บช_2024_02_04032024_New.xlsx'  #@param {type:'string'}
ecs2 = 'เจ้าของหลักประกัน_audit_2024_02.xlsx' #@param {type:'string'}
df_ecs1 = pd.read_excel(input_path + ecs1 , dtype = str)
df_ecs2 = pd.read_excel(input_path + ecs2 , dtype = str)

In [10]:
#@title ข้อมูลรวมศูนย์
# มีการให้ทุกต้นเดือนเป็น ออโต้จากฝ่าย พส.
centralize = 'รวมศูนย์_20240402.xlsx' #@param {type:'string'}
df_cen = pd.read_excel(input_path + centralize , dtype = str)

In [11]:
#@title ใส่ขอบเขตการตรวจ
start_date = '2024-01-01' #@param {type:"date"}
end_date = '2024-03-31' #@param {type:"date"}
start_date = pd.to_datetime(start_date, format = '%Y-%m-%d')
end_date = pd.to_datetime(end_date  , format = '%Y-%m-%d')

In [12]:
#@title ชื่อไฟล์ Output
file_name = 'รวมศูนย์_ผู้ค้ำประกัน_q1_67_jupyter.xlsx' #@param {type:'string'}

In [13]:
#df_b = pd.read_excel(input_path + 'Carp_ทบทวนแล้ว_110865.xlsx' , dtype = str)

# CODE

## Chose Customer and Clean Outstanding File

In [14]:
df_out1 = df_out[['รหัสลูกค้า' , 'ผู้ดูแลลูกค้า_(แยกส่วน)' , 'ผู้ดูแลลูกค้า_(ตามฝ่าย)' , 'วันที่อนุมัติ',
                           'วันที่ทบทวนล่าสุด']].drop_duplicates()

In [15]:
from datetime import datetime
def clean_date_out(a):
  if pd.isna(a) == False:
    if a[6:8] == '26':
      a = a.replace('265' , '256')
      a = a[:6] + str(int(a[6:]) - 543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '25':
      a = a[:6] + str(int(a[6:]) - 543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '20' or a[6:8] == '19':
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:7] == '3':
      a = a[:6] + str(int(a[6:]) -2*543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '00':
      a = np.nan
#     elif a[:2] == '25' and a[4] == '-':
#       a = str(int(a[:4])-543) + a[4:]
# #       a = pd.to_datetime( a , format = '%Y/%m/%d')
#       a = datetime.strptime(a, '%Y-%m-%d %H:%M:%S').date()  
#     else:
#       print(a)
  return a

In [16]:
# clean_date_out(df_out1['วันที่อนุมัติ'].iloc[592])

In [17]:
# df_out1['วันที่อนุมัติ'].iloc[590:600]

In [18]:
date_col = ['วันที่อนุมัติ' , 'วันที่ทบทวนล่าสุด']
for i in range(len(date_col)):
    df_out1[date_col[i]] = df_out1[date_col[i]].apply(lambda x : clean_date_out(x))
 # df_out1[date_col[i]] = pd.to_datetime( df_out1[date_col[i]] , format = '%d/%m/%Y')

In [19]:
for i in range(len(df_out1['วันที่อนุมัติ'])):
    if isinstance(df_out1['วันที่อนุมัติ'].iloc[i], str) == True:
        print(df_out1['รหัสลูกค้า'].iloc[i])
        print(df_out1['วันที่อนุมัติ'].iloc[i])
        print(i)

In [20]:
df_out1 = df_out1.dropna(subset = ['ผู้ดูแลลูกค้า_(แยกส่วน)'])

In [21]:
def check_date_interval(a , b , start_date , end_date):
  c = 0
  if pd.isna(a) == False:
    if (a >= start_date) and (a <= end_date):
      c +=1
  if pd.isna(b) == False:
    if (b >= start_date) and (b <= end_date):
      c +=1
  return c

In [22]:
df_out1['is_in'] = df_out1.apply(lambda x : check_date_interval(x['วันที่อนุมัติ'] , x['วันที่ทบทวนล่าสุด'] , start_date , end_date) , axis = 1)

In [23]:
df_out1.head()

,รหัสลูกค้า,ผู้ดูแลลูกค้า_(แยกส่วน),ผู้ดูแลลูกค้า_(ตามฝ่าย),วันที่อนุมัติ,วันที่ทบทวนล่าสุด,is_in
0,G009336,สาขาพระราม 4,สาขาพระราม 4,2021-04-30,2023-08-02,0
1,G009336,สาขาพระราม 4,สาขาพระราม 4,2016-09-06,2023-08-02,0
2,G009336,สาขาพระราม 4,สาขาพระราม 4,2022-09-23,2023-08-02,0
3,G009348,ส่วนคดี,ฝ่ายบริหารหนี้และนิติกรรม,2015-04-09,2023-11-30,0
5,G009525,ส่วนอุตสาหกรรม 3.1,ฝ่ายกลุ่มอุตสาหกรรม 3,2010-02-12,2023-12-22,0


In [24]:
# Clean ข้อมูลให้เหลือ 1 บรรทัดก่อนของข้อมูล outstanding
cus_list = df_out1['รหัสลูกค้า'].unique().tolist()
df_out2 = pd.DataFrame()
for i in range(len(cus_list)):
  df_t = df_out1[df_out1['รหัสลูกค้า'] == cus_list[i]]

  date1_min = min(list(df_t['วันที่อนุมัติ']))
  date1_max = max(list(df_t['วันที่อนุมัติ']))

  #date2_min = min(list(df_t['วันที่ทบทวนล่าสุด']))
  date2_max = min(list(df_t['วันที่ทบทวนล่าสุด']))

  df_t1 = df_t.head(1)
  df_t1 = df_t1.drop(columns = ['วันที่อนุมัติ' , 'วันที่ทบทวนล่าสุด'])
  df_t1['วันที่อนุมัติครั้งแรก'] = date1_min
  df_t1['วันที่อนุมัติล่าสุด'] = date1_max
  #df_t1['วันที่ทบทวนครั้งแรก'] = date2_min
  df_t1['วันที่ทบทวนล่าสุด'] = date2_max

  df_out2 = pd.concat([df_out2 , df_t1])

In [25]:
df_cusin = df_out1[df_out1['is_in'] != 0][['รหัสลูกค้า']].drop_duplicates()

In [26]:
df_out3 = pd.merge(df_out2 , df_cusin , on = 'รหัสลูกค้า')

In [27]:
print(len(df_out1) , len(df_out2) , len(df_out3))

8305 5089 1945


In [28]:
df_out4 = pd.merge(df_out3 , df_pm , on = 'รหัสลูกค้า' , how  ='left')

In [29]:
#df_out4 = df_out4.drop(columns = ['is_in'])
df_out4.head()

,รหัสลูกค้า,ผู้ดูแลลูกค้า_(แยกส่วน),ผู้ดูแลลูกค้า_(ตามฝ่าย),is_in,วันที่อนุมัติครั้งแรก,วันที่อนุมัติล่าสุด,วันที่ทบทวนล่าสุด,CUCTAXCS
0,G020405,สาขาบางนา-ตราด กม.3,สาขาบางนา-ตราด กม.3,1,2013-02-22,2021-04-30,2024-03-22,0105545080243
1,G020405,สาขาบางนา-ตราด กม.3,สาขาบางนา-ตราด กม.3,1,2013-02-22,2021-04-30,2024-03-22,0105538144240
2,G020405,สาขาบางนา-ตราด กม.3,สาขาบางนา-ตราด กม.3,1,2013-02-22,2021-04-30,2024-03-22,0105548072225
3,G020405,สาขาบางนา-ตราด กม.3,สาขาบางนา-ตราด กม.3,1,2013-02-22,2021-04-30,2024-03-22,0105562130181
4,G020774,สาขาพระราม 4,สาขาพระราม 4,1,2012-07-26,2022-02-01,2024-03-12,0105530030512


## Clean ข้อมูล ECS

In [30]:
df_ecs1.head()

,CUST_CODE,TITLE,CUST_THAI_NAME,CUST_THAI_SURNAME,CR_LIMIT_NO,FACILITY_ADD_TEXT,APPROVED_CURR_AMOUNT,EFFECTIVE_DATE,AUTHORIZE_DATE,LOG_APPROVE_DATE,...,MORTGAGEVALUE,CONTRACT_CURRENCY,COLLATERAL_NO,COLCODETYPE,COLL_TYPENAME,COLL_DESCRIPTION,APPRAISAL_VALUE,APPRAISAL_DATE,COLL_STATUS,LIM_STATUS
0,0008421,บจก.,นครการยางอุตสาหกรรม,-,400008501,"PSF(L/C,P/O,Contract)ร่วมกับรับซื้อตั๋ว D/P Sight",3142000,1997-08-11 00:00:00.0,2024-02-08 00:00:00.0,2024-02-09 00:00:00.0,...,5000000,THB,43020901,1140,ที่ดินพร้อมสิ่งปลูกสร้าง,ฉ.7000 (สปส.เลขที่ 503),3325000,2023-12-15 00:00:00.0,A,A
1,0000131,บจก.,พี.ที.ฟู้ดสตัฟส์,-,400016401,"PSF(L/C,P/O,Contract 120 วัน) ร่วมกับรับซื้อตั...",11500000,2017-10-17 00:00:00.0,2017-10-17 00:00:00.0,2022-10-07 00:00:00.0,...,0,THB,99990001,2210,นิติบุคคล,บมจ.พี.ที.พร็อพเพอร์ตี้ แมนเนจเม้นท์,0,1900-01-01 00:00:00.0,A,A
2,0000131,บจก.,พี.ที.ฟู้ดสตัฟส์,-,400016401,"PSF(L/C,P/O,Contract 120 วัน) ร่วมกับรับซื้อตั...",11500000,2017-10-17 00:00:00.0,2017-10-17 00:00:00.0,2022-10-07 00:00:00.0,...,2500000,THB,43135701,2310,บุคคลธรรมดา,นายพีรพจน์ วงศ์ภากรเกียรติ,0,1900-01-01 00:00:00.0,A,A
3,0010460,บจก.,สหกิจวิศาล,-,400074701,วงเงิน L/G ไว้ต่อการไฟฟ้าส่วนภูมิภาค,3000000,2008-09-29 00:00:00.0,2008-09-26 00:00:00.0,2013-07-24 00:00:00.0,...,11340000,THB,43153501,1220,เครื่องจักร,เครื่องจักร (43 306 605)0001:05 (5 เครื่อง),1716900,2021-05-20 00:00:00.0,A,A
4,G020405,บ/ช ร่วม,เฟรช โปรดิวส์/ไชน โปรดักส์,-,410021301,"PSF(L/C,P/O,Contract 150 วัน) ร่วมกับรับซื้อตั...",33000000,2014-10-15 00:00:00.0,2014-08-26 00:00:00.0,2023-12-13 00:00:00.0,...,33000000,THB,59015701,2210,นิติบุคคล,บจก.สกายเท็กซ์,0,1900-01-01 00:00:00.0,A,A


In [31]:
df_ecs2.head()

,COLLATERAL_NO,STATUS,OWNERSHIP_ID,FIRSTNAME,LASTNAME,PASSPORT,REGISTRATION_NO
0,43003301,A,9,เคลอินเตอร์เทรด,-,-,0105531089375
1,43003701,R,10,เคลอินเตอร์เทรด,-,-,0105531089375
2,43003801,R,11,เคลอินเตอร์เทรด,-,-,0105531089375
3,43004201,R,12,สิริพร,เกียรติเสวีกุล,-,3102401142821
4,43004401,R,13,นพดล,เกียรติเสวีกุล,-,3102401142979


In [32]:
df_e1 = df_ecs1[['CUST_CODE' , 'CR_LIMIT_NO' , 'COLL_TYPENAME' , 'COLL_DESCRIPTION' , 'COLLATERAL_NO']].drop_duplicates() #จากไฟล์ ECS บช
df_e2 = df_ecs2[['COLLATERAL_NO' , 'STATUS' , 'FIRSTNAME'  , 'LASTNAME' , 'REGISTRATION_NO']].drop_duplicates() # จากไฟล์ ECS ตส

In [33]:
df_e3 = pd.merge(df_e1 , df_e2 , on = 'COLLATERAL_NO' , how = 'left')

In [34]:
df_e3.head()

,CUST_CODE,CR_LIMIT_NO,COLL_TYPENAME,COLL_DESCRIPTION,COLLATERAL_NO,STATUS,FIRSTNAME,LASTNAME,REGISTRATION_NO
0,0008421,400008501,ที่ดินพร้อมสิ่งปลูกสร้าง,ฉ.7000 (สปส.เลขที่ 503),43020901,A,ทอง,เมืองงาว,3102401083786
1,0000131,400016401,นิติบุคคล,บมจ.พี.ที.พร็อพเพอร์ตี้ แมนเนจเม้นท์,99990001,A,พี.ที.พร็อพเพอร์ตี้ แมนเนจเม้นท์,-,0105545030548
2,0000131,400016401,บุคคลธรรมดา,นายพีรพจน์ วงศ์ภากรเกียรติ,43135701,A,พีรพจน์,วงศ์ภากรเกียรติ,3101702113322
3,0010460,400074701,เครื่องจักร,เครื่องจักร (43 306 605)0001:05 (5 เครื่อง),43153501,A,สหกิจวิศาล,-,0105532097363
4,G020405,410021301,นิติบุคคล,บจก.สกายเท็กซ์,59015701,A,สกายเท็กซ์,-,0105530057950


In [35]:
def clean_ecs_name(fname , lname):
  name1 = np.nan
  name2 = np.nan
  if pd.isna(fname) == False:
    if fname == '-':
      fname = np.nan

  if pd.isna(lname) == False:
    if lname == '-':
      lname = np.nan

  if (pd.isna(fname) == False) and (pd.isna(lname) == False):
    if isEnglish(fname) == isEnglish(lname):
      name1 = fname + lname
      name2 = np.nan
    else:
      name1 = fname
      name2 = lname
  elif (pd.isna(fname) == False) and (pd.isna(lname) == True):
    name1 = fname
    name2 = np.nan
  elif (pd.isna(fname) == True) and (pd.isna(lname) == False):
    name1 = lname
    name2 = np.nan
  elif (pd.isna(fname) == False) and (pd.isna(lname) == False):
    name1 = np.nan
    name2 = np.nan
  return name1 , name2

In [36]:
df_e3['NAME_1'] , df_e3['NAME_2'] = zip(* df_e3.apply(lambda x : clean_ecs_name( x['FIRSTNAME'] , x['LASTNAME']) , axis = 1 ))

In [37]:
df_e3[df_e3['NAME_2'].notna()]

,CUST_CODE,CR_LIMIT_NO,COLL_TYPENAME,COLL_DESCRIPTION,COLLATERAL_NO,STATUS,FIRSTNAME,LASTNAME,REGISTRATION_NO,NAME_1,NAME_2
9,0014934,420012501,บุคคลธรรมดา,นายอักบาร์ เวอร์จี (AKBAR VERJEE),43199901,A,อักบาร์ เวอร์จี,AKBAR VERJEE,0,อักบาร์ เวอร์จี,AKBAR VERJEE
10,0014934,420012601,บุคคลธรรมดา,นายอักบาร์ เวอร์จี (AKBAR VERJEE),43199901,A,อักบาร์ เวอร์จี,AKBAR VERJEE,0,อักบาร์ เวอร์จี,AKBAR VERJEE
25,0010019,440036901,บุคคลธรรมดา,นายหลาน จง เซี้ยน (LAN CHUNG-HSIEN),43157001,A,หลาน จง เซี้ยน,LAN CHUNG-HSIEN,0,หลาน จง เซี้ยน,LAN CHUNG-HSIEN
42,0031377,460025501,บุคคลธรรมดา,นายโยชิตากา โยชิกิ (YOSHITAKA YOSHIKI),44008605,A,โยชิตากา โยชิกิ,YOSITAKA YOSHIKI,0,โยชิตากา โยชิกิ,YOSITAKA YOSHIKI
53,0011883,470023301,บุคคลธรรมดา,นายย็อง-ดึก โช (YONG DEUK CHO),47027001,A,ย็อง-ดึก โช,YONG DEUK CHO,0,ย็อง-ดึก โช,YONG DEUK CHO
...,...,...,...,...,...,...,...,...,...,...,...
30823,0078402,660112101,บุคคลธรรมดา,นายลิม ฮอง เค (LIM HONG KAE),56025701,A,ลิม ฮอง เค,LIM HONG KAE,0,ลิม ฮอง เค,LIM HONG KAE
30829,0152868,660116501,บุคคลธรรมดา,นายราฮุล จันเดย์ (MR.RAHUL CHANDAY),66133801,A,ราฮุล จันเดย์,RAHUL CHANDAY,NaN,ราฮุล จันเดย์,RAHUL CHANDAY
30848,0141095,650154401,บุคคลธรรมดา,"นายฮงเปย์ วู (MR.WU, HONGPEI)",65178201,A,ฮงเปย์ วู,"WU, HONGPEI",NaN,ฮงเปย์ วู,"WU, HONGPEI"
30882,0137747,650089301,บุคคลธรรมดา,นายหวัง เห๋อหลิน (MR.WANG. HELIN),65100401,A,หวัง เห๋อหลิน,WANG. HELIN,NaN,หวัง เห๋อหลิน,WANG. HELIN


In [38]:
name_col = ['NAME_1' , 'NAME_2']
for i in range(len(name_col)):
  df_e3[name_col[i]] = df_e3[name_col[i]].apply(lambda x : clean_string(x))

In [39]:
df_e4 = pd.merge(df_e3 , df_out4 , left_on = 'CUST_CODE' , right_on = 'รหัสลูกค้า' )

In [40]:
# เลือกขอบเขตแล้ว
df_e4.head()

,CUST_CODE,CR_LIMIT_NO,COLL_TYPENAME,COLL_DESCRIPTION,COLLATERAL_NO,STATUS,FIRSTNAME,LASTNAME,REGISTRATION_NO,NAME_1,NAME_2,รหัสลูกค้า,ผู้ดูแลลูกค้า_(แยกส่วน),ผู้ดูแลลูกค้า_(ตามฝ่าย),is_in,วันที่อนุมัติครั้งแรก,วันที่อนุมัติล่าสุด,วันที่ทบทวนล่าสุด,CUCTAXCS
0,0008421,400008501,ที่ดินพร้อมสิ่งปลูกสร้าง,ฉ.7000 (สปส.เลขที่ 503),43020901,A,ทอง,เมืองงาว,3102401083786,ทองเมืองงาว,NaN,0008421,สาขาพระราม 2,สาขาพระราม 2,2,2010-12-24,2024-02-08,2024-02-08,0105528019493
1,0008421,530143401,เครื่องจักร,เครื่องจักร (เครื่องฉีดยาง 300 ตันฯ),45003501,A,นครการยางอุตสาหกรรม,-,0105528019493,นครการยางอุตสาหกรรม,NaN,0008421,สาขาพระราม 2,สาขาพระราม 2,2,2010-12-24,2024-02-08,2024-02-08,0105528019493
2,0008421,530143401,ใบฝาก หรือบัญชีเงินฝาก และอื่นๆ (EXIM),บัญชีออมทรัพย์ (0012006117),61020506,A,นครการยางอุตสาหกรรม,-,0105528019493,นครการยางอุตสาหกรรม,NaN,0008421,สาขาพระราม 2,สาขาพระราม 2,2,2010-12-24,2024-02-08,2024-02-08,0105528019493
3,0008421,400008501,บุคคลธรรมดา,นางสาย จิตตะเทวา,43132101,A,สาย,จิตตะเทวา,3102401083735,สายจิตตะเทวา,NaN,0008421,สาขาพระราม 2,สาขาพระราม 2,2,2010-12-24,2024-02-08,2024-02-08,0105528019493
4,0008421,400008501,เครื่องจักร,เครื่องจักร (เครื่องฉีดยาง 300 ตันฯ),45003501,A,นครการยางอุตสาหกรรม,-,0105528019493,นครการยางอุตสาหกรรม,NaN,0008421,สาขาพระราม 2,สาขาพระราม 2,2,2010-12-24,2024-02-08,2024-02-08,0105528019493


In [41]:
def is_debt(a):
  c = 0
  if pd.isna(a) == False:
    if ('หนี้' in a) or ('เงิน' in a):
      c = 1
  return c

In [42]:
df_e4['is_debt'] = df_e4['ผู้ดูแลลูกค้า_(ตามฝ่าย)'].apply(lambda x : is_debt(x))

In [43]:
df_e5 = df_e4[df_e4['is_debt'] == 0].drop(columns = ['is_debt'])

## Clean File รวมศูนย์

In [44]:
df_cen['NAME_C1'] , df_cen['NAME_C2'] =  zip(* df_cen.apply(lambda x : clean_ecs_name( x['NAMETH'] , x['SURNAMETH']) , axis = 1 ))

In [45]:
df_cen['NAME_C3'] , df_cen['NAME_C4'] =  zip(* df_cen.apply(lambda x : clean_ecs_name( x['NAMEEN'] , x['SURNAMEEN']) , axis = 1 ))

In [46]:
name_col = ['NAME_C1' , 'NAME_C2' , 'NAME_C3' , 'NAME_C4']
for i in range(len(name_col)):
  df_cen[name_col[i]] = df_cen[name_col[i]].apply(lambda x : clean_string(x))

In [47]:
df_cen.head()

,EXIM_CUSTID,CUSTID,ID,TYPENAME,TITLENAME,NAMETH,SURNAMETH,NAMEEN,SURNAMEEN,CUSTTYPE,...,WARNINGLIST,CREATEDON,MODIFIEDON,APPROVEDDATE,SYSTEMNAME,EXIM_TITLENAMECODENAME,NAME_C1,NAME_C2,NAME_C3,NAME_C4
0,0058283,6210497,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป/ ผู้รับผลประโยชน์...,...,ทุกครั้ง,11/16/2023 12:50:47 PM,1/15/2024 3:47:20 PM,11/16/2023 2:17:14 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
1,0062173,6205894,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,"ลูกค้า,",...,ทุกครั้ง,3/22/2022 1:52:14 PM,6/9/2023 6:42:38 AM,3/22/2022 1:54:21 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
2,0066484,6208665,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป/ ผู้รับผลประโยชน์...,...,ทุกครั้ง,3/22/2022 2:08:57 PM,1/15/2024 3:48:34 PM,3/22/2022 3:25:46 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
3,6316605,6316605,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป/ ผู้รับผลประโยชน์...,...,ทุกครั้ง,3/13/2024 3:26:22 PM,3/13/2024 5:22:45 PM,3/13/2024 5:50:24 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
4,0140569,6542384,NaN,ข้อมูลเดิม,NaN,ธนาคารอาคารสงเคราะห์แม่โขง,NaN,Mekong Housing Bank,NaN,"ลูกค้า,",...,ทุกครั้ง,12/27/2023 1:22:39 PM,12/27/2023 2:01:32 PM,12/27/2023 2:03:19 PM,LOS,NaN,ธนาคารอาคารสงเคราะห์แม่โขง,NaN,mekonghousingbank,NaN


## MERGE ครั้งละ รหัสลูกค้า

In [48]:
# Loop ที่ใช้ตรวจ
cus_list = df_e5['CUST_CODE'].unique().tolist()
name_ecs = ['NAME_1' , 'NAME_2']
name_centralize = ['NAME_C1' , 'NAME_C2' ,'NAME_C3' , 'NAME_C4']
df_notin = pd.DataFrame()
for i in range(len(cus_list)):
  df_u = df_e5[df_e5['CUST_CODE'] == cus_list[i]]
  df_c = df_cen[df_cen['EXIM_CUSTID'] == cus_list[i]]
  df_m_out = df_u.copy()
  for m in range(len(name_ecs)):
    for n in range(len(name_centralize)):
      if (len(df_m_out.dropna(subset = [name_ecs[m]])) != 0) and len(df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates()):
        df_m = fuzzy_merge(df_m_out.dropna(subset = [name_ecs[m]]) ,
                                    df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates() ,
                                    left_on = name_ecs[m] , right_on = name_centralize[n] , how = 'left')
        df_m_out = df_m[df_m[name_centralize[n]].isna()].drop(columns = name_centralize[n])
  df_notin = pd.concat([df_notin , df_m_out])

In [49]:
## สำหรับรหัส G จะยอมให้หาแต่ชื่อ

In [50]:
def id_g(a):
  c = 0
  if pd.isna(a) == False:
    if a[0] == 'G':
      c = 1
  return c

In [51]:
df_notin['is_g'] = df_notin['รหัสลูกค้า'].apply(lambda x : id_g(x))

In [52]:
df_g = df_notin[df_notin['is_g'] == 1].drop(columns = ['is_g'])

df_notg = df_notin[df_notin['is_g'] == 0].drop(columns = ['is_g'])

In [53]:
print(len(df_g) , len(df_notg))

383 174


In [54]:
cus_list = df_g['CUST_CODE'].unique().tolist()
name_ecs = ['NAME_1' , 'NAME_2']
name_centralize = ['NAME_C1' , 'NAME_C2' ,'NAME_C3' , 'NAME_C4']
df_notin1 = pd.DataFrame()
for i in range(len(cus_list)):
  df_u = df_g[df_g['CUST_CODE'] == cus_list[i]]
  df_c = df_cen.copy()
  df_m_out = df_u.copy()
  for m in range(len(name_ecs)):
    for n in range(len(name_centralize)):
      if (len(df_m_out.dropna(subset = [name_ecs[m]])) != 0) and len(df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates()):
        df_m = fuzzy_merge(df_m_out.dropna(subset = [name_ecs[m]]) ,
                                    df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates() ,
                                    left_on = name_ecs[m] , right_on = name_centralize[n] , how = 'left')
        df_m_out = df_m[df_m[name_centralize[n]].isna()].drop(columns = name_centralize[n])
  df_notin1 = pd.concat([df_notin1 , df_m_out])

In [55]:
df_ans = pd.concat([df_notg , df_notin1])

In [56]:
len(df_ans)

176

In [57]:
df_ans

,CUST_CODE,CR_LIMIT_NO,COLL_TYPENAME,COLL_DESCRIPTION,COLLATERAL_NO,STATUS,FIRSTNAME,LASTNAME,REGISTRATION_NO,NAME_1,NAME_2,รหัสลูกค้า,ผู้ดูแลลูกค้า_(แยกส่วน),ผู้ดูแลลูกค้า_(ตามฝ่าย),is_in,วันที่อนุมัติครั้งแรก,วันที่อนุมัติล่าสุด,วันที่ทบทวนล่าสุด,CUCTAXCS
0,0008741,620181001,อาคารสิ่งปลูกสร้าง,โรงงานเลขที่ 14/10 ม.5 (บน ฉ.3995-6),67003301,A,มิลแบริ่ง,NaN,0105554145719,มิลแบริ่ง,NaN,0008741,ส่วนอุตสาหกรรม 3.2,ฝ่ายกลุ่มอุตสาหกรรม 3,1,2019-11-19,2022-11-29,2024-01-05,0105533003222
1,0008741,490104201,อาคารสิ่งปลูกสร้าง,โรงงานเลขที่ 14/10 ม.5 (บน ฉ.3995-6),67003301,A,มิลแบริ่ง,NaN,0105554145719,มิลแบริ่ง,NaN,0008741,ส่วนอุตสาหกรรม 3.2,ฝ่ายกลุ่มอุตสาหกรรม 3,1,2019-11-19,2022-11-29,2024-01-05,0105533003222
2,0008741,620181101,อาคารสิ่งปลูกสร้าง,โรงงานเลขที่ 14/10 ม.5 (บน ฉ.3995-6),67003301,A,มิลแบริ่ง,NaN,0105554145719,มิลแบริ่ง,NaN,0008741,ส่วนอุตสาหกรรม 3.2,ฝ่ายกลุ่มอุตสาหกรรม 3,1,2019-11-19,2022-11-29,2024-01-05,0105533003222
1,0079520,570010301,นิติบุคคล,TTCL GAS POWER PTE.LTD.\t,57007101,A,TTCL GAS POWER PTE.LTD.,-,0,ttclgaspowerpte,NaN,0079520,ส่วนอุตสาหกรรม 2.1,ฝ่ายกลุ่มอุตสาหกรรม 2,1,2014-04-09,2014-04-09,2024-01-08,NaN
0,0054080,570041801,นิติบุคคล,นิติบุคคล (Haycarb PLC),62000106,A,Haycarb PCL,NaN,NaN,haycarbpcl,NaN,0054080,สาขาพระราม 2,สาขาพระราม 2,1,2011-10-12,2018-12-03,2024-03-22,0105536122109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12571,0110427,650018201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0110427,ส่วนอุตสาหกรรม 3.3,ฝ่ายกลุ่มอุตสาหกรรม 3,1,2022-02-15,2022-02-15,2024-01-30,NaN
12572,0143313,650208801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0143313,ส่วนอุตสาหกรรม 3.3,ฝ่ายกลุ่มอุตสาหกรรม 3,1,2022-08-08,2022-08-08,2024-01-15,NaN
12573,0104944,610139101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0104944,ส่วนอุตสาหกรรม 2.1,ฝ่ายกลุ่มอุตสาหกรรม 2,2,2024-01-30,2024-01-30,2024-01-04,0107558000229
0,G022367,590008401,นิติบุคคล,บมจ.ซีวิลเอนจิเนียริง,99990100,A,ซีวิลเอนจิเนียริง,-,0107563000169,ซีวิลเอนจิเนียริง,NaN,G022367,ส่วนอุตสาหกรรม 1.2,ฝ่ายกลุ่มอุตสาหกรรม 1,1,2016-02-02,2019-10-01,2024-01-16,0107563000169


# EXPORT FILE

In [58]:
df_ans.to_excel(output_path + file_name , index = False)